## Literature search

To be able to reproduce the results of my web search for articles for my systematic review, I used the code below.

We use PubMed for the search.

In [35]:
from pymed import PubMed
pubmed = PubMed(tool="MyTool", email="allen@dkfz-heidelberg.de")

This is just an example of how we can retrieve articles from a database search. Not part of the review.

In [71]:
query = "(kevin allen grid cells) AND ((\"2015/01/01\"[Date - Publication] : \"2024/01/01\"[Date - Publication]))"
results = pubmed.query(query, max_results=500)

In [72]:
def results_to_dict(results):
    articleList = []
    for article in results:
        articleDict = article.toDict()
        articleList.append(articleDict)
    return articleList

articleList = results_to_dict(results)

In [73]:
len(articleList)

5

In [74]:
print(f"We found {len(articleList)} articles")
articleList[0].keys()

We found 5 articles


dict_keys(['pubmed_id', 'title', 'abstract', 'keywords', 'journal', 'publication_date', 'authors', 'methods', 'conclusions', 'results', 'copyrights', 'doi', 'xml'])

In [75]:
articleList[0]

{'pubmed_id': '30222110\n26709045\n3653312\n17626218\n27852770\n20657591\n23603709\n21609831\n9700566\n28416119\n15385610\n7843305\n27498565\n20392951\n22454618\n24089485\n20505130\n28343867\n9733184\n9364077\n15965463\n28888183\n28756950\n27991898\n25024427\n8595195\n26631468\n30222110\n26176924\n16858394\n19874793\n19923286\n7996172\n30061814\n27449281\n25730672\n16675704\n11439447\n11539168\n12890795\n9151751\n9787007\n23222610\n2303851\n2303852\n8592189\n7823153\n25122712\n22983210\n26791205\n25700518\n1308170\n21451060\n19176815\n24795578\n16407560\n8604055',
 'title': 'Non-rhythmic head-direction cells in the parahippocampal region are not constrained by attractor network dynamics.',
 'abstract': 'Computational models postulate that head-direction (HD) cells are part of an attractor network integrating head turns. This network requires inputs from visual landmarks to anchor the HD signal to the external world. We investigated whether information about HD and visual landmarks is i

In [76]:
for a in articleList:
    print(a['title'])
    print(a['publication_date'])

Non-rhythmic head-direction cells in the parahippocampal region are not constrained by attractor network dynamics.
2018-09-18
Hippocampal Remapping and Its Entorhinal Origin.
2018-01-23
Impaired path integration in mice with disrupted grid cell firing.
2017-12-13
Visual landmarks sharpen grid cell metric and confer context specificity to neurons of the medial entorhinal cortex.
2016-07-28
Interspike Intervals Reveal Functionally Distinct Cell Populations in the Medial Entorhinal Cortex.
2015-08-08


We could build a list of queries and then loop through them to retrieve our paper of interests.

In [104]:
query = "(human-AI interactions) AND (radiology) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"
query = "(human-AI interactions) AND (oncology) AND((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"
query = "(human-AI interactions) AND (cancer) AND((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"


query = "(human-centered AI) AND (radiology) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"
query = "(human-centered AI) AND (oncology) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"
query = "(human-centered AI) AND (cancer) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"

query = "(human-AI interactions) OR (explainable AI) AND (cancer) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"
query = "(human-centered AI) AND (explainable AI) AND (oncology) AND ((\"2019/01/01\"[Date - Publication] : \"2024/04/01\"[Date - Publication]))"

In [105]:
results = pubmed.query(query, max_results=500)
articleList = results_to_dict(results)

In [106]:
len(articleList)

5

In [107]:
for a in articleList:
    print(a['title'])
    print(a['publication_date'])

Evaluating the clinical utility of artificial intelligence assistance and its explanation on the glioma grading task.
2024-02-08
A human-interpretable machine learning pipeline based on ultrasound to support leiomyosarcoma diagnosis.
2023-12-03
Explainable artificial intelligence (XAI): closing the gap between image analysis and navigation in complex invasive diagnostic procedures.
2022-01-28
Editorial: On the "Human" in Human-Artificial Intelligence Interaction.
2022-01-11
Mutation-based clustering and classification analysis reveals distinctive age groups and age-related biomarkers for glioma.
2021-03-01
